In [2]:
import numpy as np
from datascience import *
import pandas as pd

import os
import re
import json

d:\VNU\Lab\Tools\.venv\Lib\site-packages\datascience\maps.py:13: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Phân tích .json

In [3]:
# --- Thông số ---
json_path = "D:/VNU/Lab/Tools/wapiti_result/timesqli_report.json"
output_txt = "D:/VNU/Lab/Tools/wapiti_result/timesqli_report.txt"

# --- Xử lý ---
with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

grouped_results = {}
total_count = 0  # Đếm tổng số BenchmarkTest

for section in data:
    section_data = data[section]
    if isinstance(section_data, dict):
        for vuln_type, vuln_list in section_data.items():
            if isinstance(vuln_list, list):
                names = []
                for entry in vuln_list:
                    path = entry.get("path", "")
                    if "BenchmarkTest" in path:
                        name = path.strip("/").split("/")[-1]
                        names.append(name)
                if names:
                    if vuln_type not in grouped_results:
                        grouped_results[vuln_type] = []
                    grouped_results[vuln_type].extend(names)
                    total_count += len(names)

# --- Ghi file TXT ---
with open(output_txt, "w", encoding="utf-8") as f:
    for vuln_type, names in grouped_results.items():
        f.write(f"[{vuln_type}] ({len(names)})\n")
        for name in names:
            f.write(f"{name}\n")
        f.write("\n")

print(f"✅ Hoàn thành: Đã tìm thấy tổng cộng {total_count} BenchmarkTest và xuất ra file {output_txt}")


✅ Hoàn thành: Đã tìm thấy tổng cộng 185 BenchmarkTest và xuất ra file D:/VNU/Lab/Tools/wapiti_result/timesqli_report.txt


## Lọc url trùng kết quả 

In [88]:
folder = "D:/VNU/Lab/Tools/found_testcase"

# Lấy thư mục cha của found_testcase → đặt clean_founded_testcase cùng cấp
parent_folder = os.path.dirname(folder)
clean_folder = os.path.join(parent_folder, "clean_founded_testcase")
os.makedirs(clean_folder, exist_ok=True)

for filename in os.listdir(folder):
    if filename.endswith("_testcase.txt"):
        input_txt = os.path.join(folder, filename)
        
        # Tạo tên file cleaned: <prefix>_clean.txt trong clean_founded_testcase/
        prefix = filename.replace("_testcase.txt", "")
        output_txt = os.path.join(clean_folder, f"{prefix}_clean.txt")

        # Đọc file
        with open(input_txt, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        # Phân nhóm
        sections, current = {}, None
        for line in lines:
            line = line.strip()
            if line.startswith("[") and "]" in line:
                current = re.search(r"\[(.+?)\]", line).group(1)
                sections[current] = []
            elif line and current:
                sections[current].append(line)

        # Lọc trùng giữa Internal Server Error và các mục khác
        if "Internal Server Error" in sections:
            other_names = {n for s, names in sections.items() if s != "Internal Server Error" for n in names}
            before = len(sections["Internal Server Error"])
            sections["Internal Server Error"] = [n for n in sections["Internal Server Error"] if n not in other_names]
            after = len(sections["Internal Server Error"])
            print(f"[{filename}] Đã lọc {before - after} tên trùng khỏi Internal Server Error.")

        # Loại trùng trong chính từng mục
        for section in sections:
            sections[section] = list(dict.fromkeys(sections[section]))

        # Ghi file cleaned
        with open(output_txt, 'w', encoding='utf-8') as f:
            for section, names in sections.items():
                f.write(f"[{section}] ({len(names)})\n" + "\n".join(names) + "\n\n")

        print(f"[{filename}] ✅ Đã lọc trùng trong từng mục & ghi file sạch: {output_txt}")

print("🎉 Hoàn thành xử lý tất cả file, kết quả ở thư mục:", clean_folder)


[cmdi_testcase.txt] Đã lọc 6 tên trùng khỏi Internal Server Error.
[cmdi_testcase.txt] ✅ Đã lọc trùng trong từng mục & ghi file sạch: D:/VNU/Lab/Tools\clean_founded_testcase\cmdi_clean.txt
[ldapi_testcase.txt] ✅ Đã lọc trùng trong từng mục & ghi file sạch: D:/VNU/Lab/Tools\clean_founded_testcase\ldapi_clean.txt
[pathtraver_testcase.txt] Đã lọc 5 tên trùng khỏi Internal Server Error.
[pathtraver_testcase.txt] ✅ Đã lọc trùng trong từng mục & ghi file sạch: D:/VNU/Lab/Tools\clean_founded_testcase\pathtraver_clean.txt
[securecookie_testcase.txt] ✅ Đã lọc trùng trong từng mục & ghi file sạch: D:/VNU/Lab/Tools\clean_founded_testcase\securecookie_clean.txt
[sqli_testcase.txt] Đã lọc 151 tên trùng khỏi Internal Server Error.
[sqli_testcase.txt] ✅ Đã lọc trùng trong từng mục & ghi file sạch: D:/VNU/Lab/Tools\clean_founded_testcase\sqli_clean.txt
[xpathi_testcase.txt] ✅ Đã lọc trùng trong từng mục & ghi file sạch: D:/VNU/Lab/Tools\clean_founded_testcase\xpathi_clean.txt
[xss_testcase.txt] ✅ Đã l

## Đọc expected result 


In [7]:
# Đọc CSV và chỉ lấy 4 cột đầu
expected = Table.read_table("D:/VNU/Lab/Tools/expected_results.csv").select(0, 1, 2, 3)
# Đổi tên cột cho chuẩn: xóa khoảng trắng thừa
expected = (expected
            .relabeled("# test name", "test_name")
            .relabeled(" category", "category")
            .relabeled(" real vulnerability", "real_vulnerability")
            .relabeled(" cwe", "cwe")
           )

# Kiểm tra kết quả
expected.show(5)

test_name,category,real_vulnerability,cwe
BenchmarkTest00001,pathtraver,True,22
BenchmarkTest00002,pathtraver,True,22
BenchmarkTest00003,hash,True,328
BenchmarkTest00004,trustbound,True,501
BenchmarkTest00005,crypto,True,327


## Tách expected theo các category 

In [8]:
# Lấy tất cả category duy nhất
categories = np.unique(expected.column("category"))

# Tạo dictionary: key=category, value=bảng con
category = {cat: expected.where("category", cat) for cat in categories}

print("🔎 Danh sách các category đã tạo bảng con:")
for cat in category.keys():
    print(f"- {cat}")


🔎 Danh sách các category đã tạo bảng con:
- cmdi
- crypto
- hash
- ldapi
- pathtraver
- securecookie
- sqli
- trustbound
- weakrand
- xpathi
- xss


In [9]:
t = category["securecookie"]
t.show(5)
print("✅ True:", t.where("real_vulnerability", True).num_rows, 
      "| False:", t.where("real_vulnerability", False).num_rows)


test_name,category,real_vulnerability,cwe
BenchmarkTest00016,securecookie,False,614
BenchmarkTest00087,securecookie,True,614
BenchmarkTest00088,securecookie,False,614
BenchmarkTest00089,securecookie,False,614
BenchmarkTest00169,securecookie,True,614


✅ True: 36 | False: 31


## Đánh giá


In [92]:
import os
from datascience import Table

clean_folder = "D:/VNU/Lab/Tools/clean_founded_testcase"
results = []

for filename in os.listdir(clean_folder):
    if filename.endswith("_clean.txt"):
        filepath = os.path.join(clean_folder, filename)
        prefix = filename.replace("_clean.txt", "")

        # 1) Đọc testcase từ file cleaned → D
        with open(filepath, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        D = set()
        for line in lines:
            line = line.strip()
            if line and not line.startswith("["):
                D.add(line)

        # 2) Lấy bảng con E từ category
        if prefix not in category:
            print(f"⚠️ Bỏ qua {filename} vì không tìm thấy category['{prefix}']")
            continue

        E = category[prefix]
        TP = FP = FN = TN = 0

        for t in E.rows:
            test_name = t[0]
            real_vuln = t[2]

            if test_name in D:
                if real_vuln:
                    TP += 1
                else:
                    FP += 1
            else:
                if real_vuln:
                    FN += 1
                else:
                    TN += 1

        print(f"[{filename}] ✅ TP:{TP} FP:{FP} FN:{FN} TN:{TN}")

        results.append([prefix, TP, FP, FN, TN])

# 3) Tạo bảng tổng hợp
final_result = Table().with_columns(
    "Category", [r[0] for r in results],
    "True Positive", [r[1] for r in results],
    "False Positive", [r[2] for r in results],
    "False Negative", [r[3] for r in results],
    "True Negative", [r[4] for r in results]
)

print("\n✅ Kết quả tổng hợp confusion matrix cho tất cả file cleaned:")
final_result.show()


[cmdi_clean.txt] ✅ TP:89 FP:0 FN:37 TN:125
[ldapi_clean.txt] ✅ TP:15 FP:0 FN:12 TN:32
[pathtraver_clean.txt] ✅ TP:72 FP:0 FN:61 TN:135
[securecookie_clean.txt] ✅ TP:0 FP:0 FN:36 TN:31
[sqli_clean.txt] ✅ TP:163 FP:24 FN:109 TN:208
[xpathi_clean.txt] ✅ TP:6 FP:0 FN:9 TN:20
[xss_clean.txt] ✅ TP:137 FP:0 FN:109 TN:209

✅ Kết quả tổng hợp confusion matrix cho tất cả file cleaned:


Category,True Positive,False Positive,False Negative,True Negative
cmdi,89,0,37,125
ldapi,15,0,12,32
pathtraver,72,0,61,135
securecookie,0,0,36,31
sqli,163,24,109,208
xpathi,6,0,9,20
xss,137,0,109,209


## tính phần trăm 

In [93]:
import pandas as pd

# Chuyển final_result từ Table → DataFrame pandas
df = final_result.to_df()

# Hàm tính % cho 1 dòng
def calc_perc(row):
    total = row["True Positive"] + row["False Positive"] + row["False Negative"] + row["True Negative"]
    if total == 0:
        return pd.Series([0, 0, 0, 0])
    return pd.Series([
        round(row["True Positive"] / total * 100, 2),
        round(row["False Positive"] / total * 100, 2),
        round(row["False Negative"] / total * 100, 2),
        round(row["True Negative"] / total * 100, 2)
    ])

# Apply và tạo bảng mới chỉ với category + % columns
percent_df = df[["Category"]].copy()
percent_df[["True Positive", "False Positive", "False Negative", "True Negative"]] = df.apply(calc_perc, axis=1)

print("✅ Bảng tỷ lệ phần trăm:")
display(percent_df)


✅ Bảng tỷ lệ phần trăm:


,Category,True Positive,False Positive,False Negative,True Negative
0,cmdi,35.46,0.00,14.74,49.80
1,ldapi,25.42,0.00,20.34,54.24
2,pathtraver,26.87,0.00,22.76,50.37
3,securecookie,0.00,0.00,53.73,46.27
4,sqli,32.34,4.76,21.63,41.27
5,xpathi,17.14,0.00,25.71,57.14
6,xss,30.11,0.00,23.96,45.93


## Đúng/Sai của các testcase bị báo 500 server

In [97]:
import pandas as pd

# 1) Chuyển datascience.Table expected → pandas DataFrame
expected_df = expected.to_df()

# 2) Đọc testcase từ section [Internal Server Error] trong sqli_clean.txt
txt_path = "D:/VNU/Lab/Tools/clean_founded_testcase/sqli_clean.txt"
internal_errors = []

with open(txt_path, 'r', encoding='utf-8') as f:
    lines = f.readlines()

current_section = None
for line in lines:
    line = line.strip()
    if line.startswith("[") and "]" in line:
        current_section = line.split("]")[0][1:]
    elif line and current_section == "Internal Server Error":
        internal_errors.append(line)

print(f"✅ Đã lấy {len(internal_errors)} testcase từ Internal Server Error trong {txt_path}")

# 3) Lọc bảng chỉ chứa testcase trong Internal Server Error
col_name = expected_df.columns[0]  # cột chứa tên testcase
internal_df = expected_df[expected_df[col_name].isin(internal_errors)]

print("✅ Bảng chỉ chứa testcase Internal Server Error:")
display(internal_df)

total = len(internal_df)
true_count = internal_df[internal_df["real_vulnerability"] == True].shape[0]
false_count = internal_df[internal_df["real_vulnerability"] == False].shape[0]

true_pct = (true_count / total) * 100 if total else 0
false_pct = (false_count / total) * 100 if total else 0

print(f"✅ Trong {total} testcase Internal Server Error:")
print(f"- True vulnerabilities (có lỗi thật): {true_count} ({true_pct:.2f}%)")
print(f"- False vulnerabilities (không có lỗi): {false_count} ({false_pct:.2f}%)")


✅ Đã lấy 34 testcase từ Internal Server Error trong D:/VNU/Lab/Tools/clean_founded_testcase/sqli_clean.txt
✅ Bảng chỉ chứa testcase Internal Server Error:


,test_name,category,real_vulnerability,cwe
7,BenchmarkTest00008,sqli,True,89
51,BenchmarkTest00052,sqli,False,89
189,BenchmarkTest00190,sqli,False,89
190,BenchmarkTest00191,sqli,False,89
327,BenchmarkTest00328,sqli,True,89
328,BenchmarkTest00329,sqli,False,89
329,BenchmarkTest00330,sqli,False,89
508,BenchmarkTest00509,sqli,False,89
509,BenchmarkTest00510,sqli,True,89
588,BenchmarkTest00589,sqli,False,89


✅ Trong 34 testcase Internal Server Error:
- True vulnerabilities (có lỗi thật): 10 (29.41%)
- False vulnerabilities (không có lỗi): 24 (70.59%)
